In [14]:
import carla
import argparse

In [15]:
argparser = argparse.ArgumentParser(
    description='CARLA Manual Control Client')
argparser.add_argument(
    '-v', '--verbose',
    action='store_true',
    dest='debug',
    help='print debug information')
argparser.add_argument(
    '--host',
    metavar='H',
    default='127.0.0.1',
    help='IP of the host server (default: 127.0.0.1)')
argparser.add_argument(
    '-p', '--port',
    metavar='P',
    default=2000,
    type=int,
    help='TCP port to listen to (default: 2000)')
argparser.add_argument(
    '-a', '--autopilot',
    action='store_true',
    help='enable autopilot')
argparser.add_argument(
    '--res',
    metavar='WIDTHxHEIGHT',
    default='1280x720',
    help='window resolution (default: 1280x720)')
argparser.add_argument(
    '--filter',
    metavar='PATTERN',
    default='vehicle.*',
    help='actor filter (default: "vehicle.*")')
argparser.add_argument(
    '--rolename',
    metavar='NAME',
    default='hero',
    help='actor role name (default: "hero")')
argparser.add_argument(
    '--gamma',
    default=2.2,
    type=float,
    help='Gamma correction of the camera (default: 2.2)')
argparser.add_argument(
    '--map',
    metavar='NAME',
    default='Town04',
    help='simulation map (default: "Town04")')
argparser.add_argument(
    '--spawn_x',
    metavar='x',
    default='-16.75', #town04 = -16.75
    help='x position to spawn the agent')
argparser.add_argument(
    '--spawn_y',
    metavar='y',
    default='-223.55', #town04 = -223.55
    help='y position to spawn the agent')
argparser.add_argument(
    '--random_spawn',        
    metavar='RS',
    default='0',
    type=int,
    help='Random spawn agent')
argparser.add_argument(
    '--vehicle_id',
    metavar='NAME',
    # default='vehicle.jeep.wrangler_rubicon',
    default='vehicle.tesla.model3',
    help='vehicle to spawn, available options : vehicle.audi.a2 vehicle.audi.tt vehicle.carlamotors.carlacola vehicle.citroen.c3 vehicle.dodge_charger.police vehicle.jeep.wrangler_rubicon vehicle.yamaha.yzf vehicle.nissan.patrol vehicle.gazelle.omafiets vehicle.bh.crossbike vehicle.ford.mustang vehicle.bmw.isetta vehicle.audi.etron vehicle.harley-davidson.low rider vehicle.mercedes-benz.coupe vehicle.bmw.grandtourer vehicle.toyota.prius vehicle.diamondback.century vehicle.tesla.model3 vehicle.seat.leon vehicle.lincoln.mkz2017 vehicle.kawasaki.ninja vehicle.volkswagen.t2 vehicle.nissan.micra vehicle.chevrolet.impala vehicle.mini.cooperst')
argparser.add_argument(
    '--vehicle_wheelbase',
    metavar='NAME',
    type=float,
    default='2.89',
    help='vehicle wheelbase used for model predict control')
argparser.add_argument(
    '--waypoint_resolution',
    metavar='WR',
    default='0.5',
    type=float,
    help='waypoint resulution for control')
argparser.add_argument(
    '--waypoint_lookahead_distance',
    metavar='WLD',
    default='5.0',
    type=float,
    help='waypoint look ahead distance for control')
argparser.add_argument(
    '--desired_speed',
    metavar='SPEED', 
    default='20',
    type=float,
    help='desired speed for highway driving')
argparser.add_argument(
    '--control_mode',
    metavar='CONT',
    default='MPC',
    help='Controller')
argparser.add_argument(
    '--planning_horizon',
    metavar='HORIZON',
    type=int,
    default='5',
    help='Planning horizon for MPC')
argparser.add_argument(
    '--time_step',
    metavar='DT',
    default='0.15',
    type=float,
    help='Planning time step for MPC')
argparser.add_argument(
    '--FPS',
    metavar='FPS',
    default='15',
    type=int,
    help='Frame per second for simulation')

args = argparser.parse_args()

In [16]:
client = carla.Client(args.host, args.port)
client.set_timeout(100.0)
carla_world = client.load_world(args.map)


In [4]:
carla_world = client.get_world()

In [17]:
blueprint_library = carla_world.get_blueprint_library()
vehicle_blueprint = blueprint_library.filter('*vehicle*')
walker_blueprint = blueprint_library.filter('*walker.*')

spawn_location = carla.Location()
spawn_location.x = float(args.spawn_x)
spawn_location.y = float(args.spawn_y)
spawn_waypoint = carla_world.get_map().get_waypoint(spawn_location)
spawn_transform = spawn_waypoint.transform
spawn_transform.location.z = 1.0
player = carla_world.try_spawn_actor(vehicle_blueprint.filter('model3')[0], spawn_transform)

In [8]:
physic_control = player.get_physics_control()

In [9]:
physic_control.use_sweep_wheel_collision = True

In [10]:
wheels = physic_control.wheels
center_of_mass = physic_control.center_of_mass

In [6]:
import numpy as np

In [11]:
front_left_wheel = wheels[0]
front_right_wheel = wheels[1]
back_left_wheel = wheels[2]
back_right_wheel = wheels[3]
front_x = (front_left_wheel.position.x + front_right_wheel.position.x) / 2.0
front_y = (front_left_wheel.position.y + front_right_wheel.position.y) / 2.0
front_z = (front_left_wheel.position.z + front_right_wheel.position.z) / 2.0
back_x = (back_left_wheel.position.x + back_right_wheel.position.x) / 2.0
back_y = (back_left_wheel.position.y + back_right_wheel.position.y) / 2.0
back_z = (back_left_wheel.position.z + back_right_wheel.position.z) / 2.0
l = np.sqrt( (front_x - back_x)**2 + (front_y - back_y)**2 + (front_z - back_z)**2  ) / 100.0

In [12]:
l - center_of_mass.x

2.554633582093911

In [41]:
center_of_mass.y

0.0

In [18]:
# creating parked vehicles

parking_position = carla.Transform(player.get_transform().location + carla.Location(-0.5, 35, 0.5), 
                        carla.Rotation(0,90,0))
parked_vehicle = carla_world.spawn_actor(vehicle_blueprint.filter('model3')[0], parking_position)

carla_world.tick()

380

In [19]:
import math
# EGO information
velocity_vec = player.get_velocity()
current_transform = player.get_transform()
current_location = current_transform.location
current_x = current_location.x
current_y = current_location.y
current_speed = math.sqrt(velocity_vec.x**2 + velocity_vec.y**2 + velocity_vec.z**2)


#Parked vehicle information
parked_transform = parked_vehicle.get_transform()
parked_location = parked_transform.location
parked_x = parked_location.x
parked_y = parked_location.y

x_dist = current_x -parked_x
y_dist = current_y -parked_y

In [20]:
current_x

-16.892932891845703

In [21]:
parked_x

-17.392932891845703

In [22]:
dist = np.sqrt((parked_y-current_y)**2 + (current_x-parked_x)**2)

In [23]:
dist

35.00357124637428

In [25]:
velocity_park = parked_vehicle.get_velocity()

In [26]:
parked_speed = math.sqrt(velocity_park.x**2 + velocity_park.y**2 + velocity_park.z**2)

In [27]:
parked_speed

0.0